In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import IPython.display
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [ ]:
df = pd.read_csv("df_filtered.csv")
df.head()

In [ ]:
df["VARIABLE"].unique()

TODO: Rename to columns to `pH`, `Water Temperature`, etc.

Check to see if there are any outliers in the data that we may need to clean up.

In [ ]:
for variable in df['VARIABLE'].unique():
    units = df[df['VARIABLE'] == variable]['UNIT_UNITE'].unique()[0]
    px.scatter(df[df['VARIABLE'] == variable]['VALUE'], title=variable, labels={'value':units}).show()
    #px.scatter(df[df['VARIABLE'] == variable], y='VALUE', color='LOCATION', title=variable, labels={'value':units}).show()

## Data Cleaning

It looks like there are some pH values that are outside of the allowed range:

In [ ]:
df[(df['VARIABLE']=='PH') & (~df['VALUE'].between(1,14))]

Let's drop the rows with values outside the range 1-14 for PH.

In [ ]:
indices = df[(df['VARIABLE']=='PH') & (~df['VALUE'].between(1,14))].index
df.drop(indices, inplace=True)

# check that there are no more rows with invalid values
print(df[df['VARIABLE'] == 'PH']['VALUE'].max())
print(df[df['VARIABLE'] == 'PH']['VALUE'].min())

In [ ]:
px.scatter(df[df['VARIABLE'] == 'PH']['VALUE'])

As well, we will drop any water temperature readings that are below 0 °C.

In [ ]:
indices = df[(df['VARIABLE']=='TEMPERATURE WATER') & (df['VALUE']<0)].index
df.drop(indices, inplace=True)

print(df[df['VARIABLE'] == 'TEMPERATURE WATER']['VALUE'].max())
print(df[df['VARIABLE'] == 'TEMPERATURE WATER']['VALUE'].min())

And drop any dissolved oxygen values that above 20 mg/L.

In [ ]:
indices = df[(df['VARIABLE']=='OXYGEN DISSOLVED') & (df['VALUE']>20)].index
df.drop(indices, inplace=True)

print(df[df['VARIABLE'] == 'OXYGEN DISSOLVED']['VALUE'].max())
print(df[df['VARIABLE'] == 'OXYGEN DISSOLVED']['VALUE'].min())

## Create Scatterplots on Maps

In [ ]:
# 'open-street-map', 'white-bg', 'carto-positron', 'carto-darkmatter', 'stamen- terrain', 'stamen-toner', 'stamen-watercolor'
for variable in df['VARIABLE'].unique():
    px.scatter_mapbox(df[df['VARIABLE'] == variable], 
                        lat='LATITUDE', 
                        lon='LONGITUDE', 
                        color='LOCATION', 
                        animation_frame='YEAR', 
                        title=variable, 
                        mapbox_style='carto-positron', 
                        zoom=2, 
                        height=600,
                        size='VALUE', 
                        #size = (d-df[df['VARIABLE']=='PH']['VALUE'].min())/16+1,
                        size_max=20, 
                        opacity=0.5
                        ).show()

In [ ]:
variable = 'TEMPERATURE WATER'
px.scatter_mapbox(df[df['VARIABLE'] == variable], 
                        lat='LATITUDE', 
                        lon='LONGITUDE', 
                        color='LOCATION', 
                        animation_frame='YEAR', 
                        title=variable,
                        mapbox_style='carto-positron', 
                        zoom=2, 
                        height=600,
                        size='VALUE', 
                        #size = (d-df[df['VARIABLE']=='PH']['VALUE'].min())/16+1,
                        size_max=20, 
                        opacity=0.5
                        ).show()

In [ ]:
# Trying to create scatter_mapbox subplots with Plotly Express (failed)
fig = make_subplots(rows=2, cols=2,
                   specs=[[{"type":"mapbox"}, {"type":"mapbox"}],
                         [{"type":"mapbox"}, {"type":"mapbox"}]])

fig.add_trace(px.scatter_mapbox(df[df["VARIABLE"] == "PH"],
                                lat=df[df["VARIABLE"] == "PH"]["LATITUDE"],
                                lon=df[df["VARIABLE"] == "PH"]["LONGITUDE"],
                                color=df[df["VARIABLE"] == "PH"]["LOCATION"],
                                animation_frame=df[df["VARIABLE"] == "PH"]["YEAR"],
                                size=df[df["VARIABLE"] == "PH"]["VALUE"].abs().to_list(),
                                size_max=50,
                                opacity=0.5), 1, 1)
'''
fig.add_trace(px.scatter_mapbox(df[df["VARIABLE"] == "TEMPERATURE WATER"],
                                lat="LATITUDE",
                                lon="LONGITUDE",
                                color="LOCATION",
                                animation_frame="YEAR",
                                size=df["VALUE"].abs().to_list(),
                                size_max=50,
                                opacity=0.5), 1, 2)

fig.add_trace(px.scatter_mapbox(df[df["VARIABLE"] == "SULPHATE DISSOLVE"],
                                lat="LATITUDE",
                                lon="LONGITUDE",
                                color="LOCATION",
                                animation_frame="YEAR",
                                size=df["VALUE"].abs().to_list(),
                                size_max=50,
                                opacity=0.5), 2, 1)

fig.add_trace(px.scatter_mapbox(df[df["VARIABLE"] == "SULPHATE DISSOLVE"],
                                lat="LATITUDE",
                                lon="LONGITUDE",
                                color="LOCATION",
                                animation_frame="YEAR",
                                size=df["VALUE"].abs().to_list(),
                                size_max=50,
                                opacity=0.5), 2, 2)
'''
fig.update_layout(mapbox={"style": "carto-positron", "zoom": 4}, margin={"t":0,"b":0,"l":0,"r":0})

fig.show()

In [ ]:
# trying to add traces from each figure.data

fig = make_subplots(rows=1, cols=2,
                   specs=[[{"type":"mapbox"}, {"type":"mapbox"}]])

foo = px.scatter_mapbox(df[df["VARIABLE"] == "PH"],
                                lat=df[df["VARIABLE"] == "PH"]["LATITUDE"],
                                lon=df[df["VARIABLE"] == "PH"]["LONGITUDE"],
                                color=df[df["VARIABLE"] == "PH"]["LOCATION"],
                                animation_frame=df[df["VARIABLE"] == "PH"]["YEAR"],
                                size=df[df["VARIABLE"] == "PH"]["VALUE"].abs().to_list(),
                                size_max=50,
                                opacity=0.5)
for trace in foo.data:
    fig.add_trace(trace, row=1, col=1)

bar = px.scatter_mapbox(df[df["VARIABLE"] == "PH"],
                                lat=df[df["VARIABLE"] == "PH"]["LATITUDE"],
                                lon=df[df["VARIABLE"] == "PH"]["LONGITUDE"],
                                color=df[df["VARIABLE"] == "PH"]["LOCATION"],
                                animation_frame=df[df["VARIABLE"] == "PH"]["YEAR"],
                                size=df[df["VARIABLE"] == "PH"]["VALUE"].abs().to_list(),
                                size_max=50,
                                opacity=0.5)
for trace in bar.data:
    fig.add_trace(trace, row=1, col=2)

fig.show()

In [ ]:
# Trying to create scatter_mapbox subplots with Plotly Graph Object (Somewhat failed... code runs)
fig = make_subplots(
    rows=1, cols=2, subplot_titles=('Title1', 'Title2'),
    specs=[[{"type": "mapbox"}, {"type": "mapbox"}]]
)

fig.add_trace(go.Scattermapbox(lat=df[df["VARIABLE"]=="PH"]["LATITUDE"],
                              lon=df[df["VARIABLE"]=="PH"]["LONGITUDE"],
                              mode="markers",
                              hoverinfo="text",
                               text=df[df["VARIABLE"]=="PH"]["LOCATION"].astype(str) + "<br>" + "<b>Value: </b>" + df["VALUE"].astype(str),
                               marker={
                                      "size":df[df["VARIABLE"]=="PH"]["VALUE"].abs().to_list(),
                                      "opacity":0.6
                                  }), row=1, col=1)

fig.add_trace(go.Scattermapbox(lat=df[df["VARIABLE"]=="TEMPERATURE WATER"]["LATITUDE"],
                              lon=df[df["VARIABLE"]=="TEMPERATURE WATER"]["LONGITUDE"],
                              mode="markers",
                              hoverinfo="text",
                               text=df[df["VARIABLE"]=="TEMPERATURE WATER"]["LOCATION"].astype(str) + "<br>" + "<b>Value: </b>" + df["VALUE"].astype(str),
                               marker={
                                      "size":df[df["VARIABLE"]=="TEMPERATURE WATER"]["VALUE"].abs().to_list(),
                                      "opacity":0.6
                                  }), row=1, col=2)

# fig.update_layout(mapbox = {'style': "carto-positron",
#                             'center': {'lon':-106, 'lat': 56},
#                             'zoom': 3})

fig.update_mapboxes(
    bearing=0,
    style="carto-positron",
    center=dict(lat=56,lon=-106),
    pitch=0,
    zoom=5)
   

fig.show()

In [ ]:
# Trying to create scatter_mapbox subplots with Plotly Graph Object (Somewhat failed... code runs)
fig = go.Figure()

fig.add_trace(go.Scattermapbox(lat=df[df["VARIABLE"]=="PH"]["LATITUDE"],
                              lon=df[df["VARIABLE"]=="PH"]["LONGITUDE"],
                              mode="markers",
                              hoverinfo="text",
                               text=df[df["VARIABLE"]=="PH"]["LOCATION"].astype(str) + "<br>" + "<b>Value: </b>" + df["VALUE"].astype(str),
                               marker={
                                      "size":df[df["VARIABLE"]=="PH"]["VALUE"].abs().to_list(),
                                      "opacity":0.6
                                  }, subplot="mapbox"))

fig.add_trace(go.Scattermapbox(lat=df[df["VARIABLE"]=="TEMPERATURE WATER"]["LATITUDE"],
                              lon=df[df["VARIABLE"]=="TEMPERATURE WATER"]["LONGITUDE"],
                              mode="markers",
                              hoverinfo="text",
                               text=df[df["VARIABLE"]=="TEMPERATURE WATER"]["LOCATION"].astype(str) + "<br>" + "<b>Value: </b>" + df["VALUE"].astype(str),
                               marker={
                                      "size":df[df["VARIABLE"]=="TEMPERATURE WATER"]["VALUE"].abs().to_list(),
                                      "opacity":0.6
                                  }, subplot="mapbox"))

# fig.update_layout(mapbox = {'style': "carto-positron",
#                             'center': {'lon':-106, 'lat': 56},
#                             'zoom': 3})


fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        style='open-street-map',
        domain={'x': [0, 0.4], 'y': [0, 1]},
        bearing=0,
        center=dict(
            lat=45,
            lon=-73
        ),
        pitch=0,
        zoom=5
    ),
        mapbox2=dict(
        style='open-street-map',
        domain={'x': [0.6, 1.0], 'y': [0, 1]},
        bearing=0,
        center=dict(
            lat=45,
            lon=-73
        ),
        pitch=0,
        zoom=5
    )
)

fig.show()

In [ ]:
def make_viz(df):
    years = df["YEAR"].unique().tolist()

    hovertemplated="<br>".join([
        "<b>Province:  %{customdata[0]}</b>",
        "<br>Longitude: %{customdata[2]}",
        "Latitude: %{customdata[1]}",
        "Location: %{customdata[3]}",
        "Variable: %{customdata[4]}",
        "Value: %{customdata[5]}",
    ])

    frames = [{
        "name":"frame_{}".format(year),
        "data":[{
            "type":"scattermapbox",
            "lat":df.loc[df["YEAR"] == year]["LATITUDE"],
            "lon":df.loc[df["YEAR"] == year]["LONGITUDE"],
            "marker":go.scattermapbox.Marker(
                size=df.loc[df["YEAR"] == year]["VALUE"].abs().to_list(),
                color=df.loc[df["YEAR"] == year]["COLOR"]
            ),
            "customdata":np.stack((df.loc[df["YEAR"] == year]["PROV_TERR"],
                                   df.loc[df["YEAR"] == year]["LATITUDE"],
                                   df.loc[df["YEAR"] == year]["LONGITUDE"],
                                  df.loc[df["YEAR"] == year]["LOCATION"],
                                  df.loc[df["YEAR"] == year]["VARIABLE"],
                                  df.loc[df["YEAR"] == year]["VALUE"]), axis=1),
            "hovertemplate": hovertemplated + "<extra></extra>"
        }],
    } for year in years]

    sliders = [{
    "transition":{"duration":0},
    "x":0.08,
    "len":0.9,
    "currentvalue":{"font":{"size":15},"prefix":"Year: ", "visible":True, "xanchor":"center"},
    "steps":[
        {
            "label":year,
            "method":"animate",
            "args":[
                ["frame_{}".format(year)],
                {'mode':'immediate', 'frame':{'duration':100, 'redraw': True}, 'transition':{'duration':50}}
            ],
        } for year in years]
    }]

    play_button = [{
    'type':'buttons',
    'showactive':True,
    'x':0.045, 'y':-0.1,
    'buttons':[{ 
        'label':'►',
        'method':'animate',
        'args':[
            None,
            {
                'frame':{'duration':100, 'redraw':True},
                'transition':{'duration':50},
                'fromcurrent':True,
                'mode':'immediate'}
            ]
        }]
    }]

    data = frames[0]["data"]

    layout = go.Layout(
        sliders=sliders,
        updatemenus=play_button,
        mapbox = {
            'style': "carto-positron",
            'center': {'lon':-106, 'lat': 56},
            'zoom': 3}
    )

    myfig = go.Figure(data=data, layout=layout, frames=frames)
    myfig.update_layout(width=800, height=500, margin=dict(l=20, r=20, t=0, b=20))
    myfig.show()

output = widgets.Output()
variables = widgets.Dropdown(options=sorted(df.VARIABLE.unique()), 
                             value=df["VARIABLE"].unique().tolist()[0], 
                             description="Variable: ")

def water_viz(change):
    global df
    df = df.loc[df["VARIABLE"] == variables.value]

    display(variables)
    make_viz(df)
    IPython.display.clear_output(wait=True)

    
variables.observe(water_viz, names="value")

In [ ]:
# Creating sldier + dropdown with a widget
water_viz("")